In [84]:
import keras
from keras import layers

import tensorflow as tf
from tensorflow.python.ops import math_ops 

import math
import numpy as np
import pandas as pd
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from functools import partial

In [85]:
#df = pd.read_csv('../dataset/datasets_for_training/auto_label_data_augmented_2.csv')

#print(df.head())

#df2 = pd.read_csv('../dataset/datasets_for_pandas/tt_dataset_agg1.csv')
#print(df2.head())

#df3 = pd.read_csv('../dataset/datasets_for_training/tt_full_dataset.csv')
#print(df3.head())
CSV_HEADER = [
  "accelerometerXAxis",
  "accelerometerYAxis",
  "accelerometerZAxis",
  "gyroscopeXAxis",
  "gyroscopeYAxis",
  "gyroscopeZAxis",
  "drivingType",
  "weightSum",
  "label",
] 


df = pd.read_csv('../dataset/datasets_for_training/tt_final_labels_with_cat_feature_weight_column_from_accsum_with_att_accsum_normalized.csv')
print(df.head())

#devide the dataset into training and testing
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

print(train_dataset.head())
print(test_dataset.head())

print(train_dataset.shape)
print(test_dataset.shape)

train_data_file = 'driving_train_data.csv'
test_data_file = 'driving_test_data.csv'

train_dataset.to_csv(train_data_file, index=False, header=False)
test_dataset.to_csv(test_data_file, index=False, header=False)


   accelerometerXAxis  accelerometerYAxis  accelerometerZAxis  gyroscopeXAxis  \
0            0.566538            0.693148            0.411252        0.663333   
1            0.566232            0.668118            0.443463        0.721667   
2            0.508809            0.647404            0.425501        0.725000   
3            0.464473            0.446312            0.471448        0.615000   
4            0.406386            0.503381            0.457353        0.670000   

   gyroscopeYAxis  gyroscopeZAxis   drivingType  weightSum   label  
0        0.630162        0.788462  Accelerating   3.497873  Normal  
1        0.565530        0.705128  Accelerating   3.478943  Normal  
2        0.554758        0.621795  Accelerating   2.810447  Normal  
3        0.657092        0.538462  Accelerating   0.640633  Normal  
4        0.692998        0.475641  Accelerating   0.911751  Normal  
     accelerometerXAxis  accelerometerYAxis  accelerometerZAxis  \
306            0.356358         

In [86]:
NUMERIC_FEATURE_NAMES = [
    "accelerometerXAxis",
    "accelerometerYAxis",
    "accelerometerZAxis",
    "gyroscopeXAxis",
    "gyroscopeYAxis",
    "gyroscopeZAxis",
]

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "drivingType": sorted(list(train_dataset["drivingType"].unique())),
}
WEIGHT_COLUMN_NAME = "weightSum"
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERIC_FEATURE_NAMES + [WEIGHT_COLUMN_NAME] else ["NA"]
    for feature_name in CSV_HEADER
]

# The name of the target feature.
TARGET_FEATURE_NAME = "label"
# A list of the labels of the target features.
TARGET_LABELS = ["Aggressive", "Normal"]

In [99]:
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 265
NUM_EPOCHS = 30

NUM_TRANSFORMER_BLOCKS = 3  # Number of transformer blocks.
NUM_HEADS = 4  # Number of attention heads.
EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [
    2,
    1,
]  # MLP hidden layer units, as factors of the number of inputs.
NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

In [88]:
target_label_lookup = layers.StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)


def prepare_example(features, target):
    target_index = target_label_lookup(target)
    weights = features.pop(WEIGHT_COLUMN_NAME)
    return features, target_index, weights


lookup_dict = {}
for feature_name in CATEGORICAL_FEATURE_NAMES:
    vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
    # Create a lookup to convert a string values to an integer indices.
    # Since we are not using a mask token, nor expecting any out of vocabulary
    # (oov) token, we set mask_token to None and num_oov_indices to 0.
    lookup = layers.StringLookup(
        vocabulary=vocabulary, mask_token=None, num_oov_indices=0
    )
    lookup_dict[feature_name] = lookup


def encode_categorical(batch_x, batch_y, weights):
    for feature_name in CATEGORICAL_FEATURE_NAMES:
        batch_x[feature_name] = lookup_dict[feature_name](batch_x[feature_name])

    return batch_x, batch_y, weights


def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    dataset = (
        tf_data.experimental.make_csv_dataset(
            csv_file_path,
            batch_size=batch_size,
            column_names=CSV_HEADER,
            column_defaults=COLUMN_DEFAULTS,
            label_name=TARGET_FEATURE_NAME,
            num_epochs=1,
            header=False,
            na_value="?",
            shuffle=shuffle,
        )
        .map(prepare_example, num_parallel_calls=tf_data.AUTOTUNE, deterministic=False)
        .map(encode_categorical)
    )
    return dataset.cache()

In [89]:
def run_experiment(
    model,
    train_data_file,
    test_data_file,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
):
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
    )

    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)

    print("Start training the model...")
    history = model.fit(
        train_dataset, epochs=num_epochs, validation_data=validation_dataset
    )
    print("Model training finished")

    _, accuracy = model.evaluate(validation_dataset, verbose=0)

    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")

    return history

In [90]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="float32"
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="int32"
            )
    return inputs

In [91]:
def encode_inputs(inputs, embedding_dims):
    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            # Create a lookup to convert a string values to an integer indices.
            # Since we are not using a mask token, nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and num_oov_indices to 0.

            # Convert the string input values into integer indices.

            # Create an embedding layer with the specified dimensions.
            embedding = layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(inputs[feature_name])
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:
            # Use the numerical features as-is.
            #numerical_feature = ops.expand_dims(inputs[feature_name], -1)
            numerical_feature = tf.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list

In [92]:
def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):
    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer())
        mlp_layers.append(layers.Dense(units, activation=activation))
        mlp_layers.append(layers.Dropout(dropout_rate))

    return keras.Sequential(mlp_layers, name=name)

In [93]:
def create_baseline_model(
    embedding_dims, num_mlp_blocks, mlp_hidden_units_factors, dropout_rate
):
    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    # Concatenate all features.
    features = layers.concatenate(
        encoded_categorical_feature_list + numerical_feature_list
    )
    # Compute Feedforward layer units.
    feedforward_units = [features.shape[-1]]

    # Create several feedforwad layers with skip connections.
    for layer_idx in range(num_mlp_blocks):
        features = create_mlp(
            hidden_units=feedforward_units,
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=layers.LayerNormalization,
            name=f"feedforward_{layer_idx}",
        )(features)

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization,
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_model = create_baseline_model(
    embedding_dims=EMBEDDING_DIMS,
    num_mlp_blocks=NUM_MLP_BLOCKS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", baseline_model.count_params())
keras.utils.plot_model(baseline_model, show_shapes=True, rankdir="LR")

Total model weights: 3421
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [100]:
history = run_experiment(
    model=baseline_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)

Start training the model...
Epoch 1/50
      1/Unknown - 1s 618ms/step - loss: -0.4256 - accuracy: 0.6226WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 1s 93ms/step - loss: -0.2493 - accuracy: 0.6339 - val_loss: -0.2835 - val_accuracy: 0.7143
Epoch 2/50
1/3 [=========>....................] - ETA: 0s - loss: -0.5014 - accuracy: 0.6226WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 3ms/step - loss: -0.2938 - accuracy: 0.6089 - val_loss: -0.4315 - val_accuracy: 0.7143
Epoch 3/50
1/3 [=========>....................] - ETA: 0s - loss: -0.3616 - accuracy: 0.6226WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 10ms/step - loss: -0.2184 - accuracy: 0.6339 - val_loss: -0.5690 - val_accuracy: 0.7143
Epoch 4/50
1/3 [=========>....................] - ETA: 0s - loss: -0.6900 - accuracy: 0.6113WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 6ms/step - loss: -0.4079 - accuracy: 0.6321 - val_loss: -0.7034 - val_accuracy: 0.7143
Epoch 5/50
1/3 [=========>....................] - ETA: 0s - loss: -0.6813 - accuracy: 0.6491WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


...

3/3 [==============================] - 0s 4ms/step - loss: -11.0592 - accuracy: 0.7125 - val_loss: -17.0350 - val_accuracy: 0.7500
Epoch 48/50
1/3 [=========>....................] - ETA: 0s - loss: -13.1742 - accuracy: 0.7132WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 4ms/step - loss: -11.0180 - accuracy: 0.7143 - val_loss: -17.6767 - val_accuracy: 0.7500
Epoch 49/50
1/3 [=========>....................] - ETA: 0s - loss: -14.3857 - accuracy: 0.7170WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 4ms/step - loss: -12.0337 - accuracy: 0.7036 - val_loss: -18.1386 - val_accuracy: 0.7500
Epoch 50/50
1/3 [=========>....................] - ETA: 0s - loss: -12.7707 - accuracy: 0.7321WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 3ms/step - loss: -11.0999 - accuracy: 0.7196 - val_loss: -18.5535 - val_accuracy: 0.7286
Model training finished


Validation accuracy: 72.86%


In [101]:
def create_tabtransformer_classifier(
    num_transformer_blocks,
    num_heads,
    embedding_dims,
    mlp_hidden_units_factors,
    dropout_rate,
    use_column_embedding=False,
):
    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    # Stack categorical feature embeddings for the Tansformer.
    #encoded_categorical_features = ops.stack(encoded_categorical_feature_list, axis=1)
    encoded_categorical_features = tf.stack(encoded_categorical_feature_list, axis=1)
    # WE DONT HAVE ANY CATEGORICAL FEATURES IN THE DATASET
    #encoded_categorical_features = None 


    # Concatenate numerical features.
    numerical_features = layers.concatenate(numerical_feature_list)

    # Add column embedding to categorical feature embeddings.
    if use_column_embedding:
        num_columns = encoded_categorical_features.shape[1]
        column_embedding = layers.Embedding(
            input_dim=num_columns, output_dim=embedding_dims
        )
        #column_indices = ops.arange(start=0, stop=num_columns, step=1)
        column_indices = tf.range(start=0, limit=num_columns, delta=1)
        encoded_categorical_features = encoded_categorical_features + column_embedding(
            column_indices
        )

    # Create multiple layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)
        # Skip connection 1.
        x = layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )
        # Layer normalization 1.
        x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)
        # Feedforward.
        feedforward_output = create_mlp(
            hidden_units=[embedding_dims],
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=partial(
                layers.LayerNormalization, epsilon=1e-6
            ),  # using partial to provide keyword arguments before initialization
            name=f"feedforward_{block_idx}",
        )(x)
        # Skip connection 2.
        x = layers.Add(name=f"skip_connection2_{block_idx}")([feedforward_output, x])
        # Layer normalization 2.
        encoded_categorical_features = layers.LayerNormalization(
            name=f"layer_norm2_{block_idx}", epsilon=1e-6
        )(x)

    # Flatten the "contextualized" embeddings of the categorical features.
    categorical_features = layers.Flatten()(encoded_categorical_features)
    # Apply layer normalization to the numerical features.
    numerical_features = layers.LayerNormalization(epsilon=1e-6)(numerical_features)
    # Prepare the input for the final MLP block.
    features = layers.concatenate([categorical_features, numerical_features])

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization,
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


tabtransformer_model = create_tabtransformer_classifier(
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    num_heads=NUM_HEADS,
    embedding_dims=EMBEDDING_DIMS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", tabtransformer_model.count_params())
keras.utils.plot_model(tabtransformer_model, show_shapes=True, rankdir="LR")

Total model weights: 16349
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [102]:
history = run_experiment(
    model=tabtransformer_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)

Start training the model...
Epoch 1/50
      1/Unknown - 2s 2s/step - loss: 1.7192 - accuracy: 0.5132WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 2s 181ms/step - loss: 1.4546 - accuracy: 0.5143 - val_loss: 0.0916 - val_accuracy: 0.7071
Epoch 2/50
1/3 [=========>....................] - ETA: 0s - loss: 0.7339 - accuracy: 0.5019WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 7ms/step - loss: 0.6128 - accuracy: 0.5250 - val_loss: 0.1157 - val_accuracy: 0.7143
Epoch 3/50
1/3 [=========>....................] - ETA: 0s - loss: 0.2259 - accuracy: 0.6264WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 7ms/step - loss: 0.1515 - accuracy: 0.6268 - val_loss: 0.2928 - val_accuracy: 0.7143
Epoch 4/50
1/3 [=========>....................] - ETA: 0s - loss: -0.0370 - accuracy: 0.6264WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 7ms/step - loss: -0.0625 - accuracy: 0.6107 - val_loss: 0.3890 - val_accuracy: 0.7143
Epoch 5/50
1/3 [=========>....................] - ETA: 0s - loss: -0.1356 - accuracy: 0.6415WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


...

3/3 [==============================] - 0s 7ms/step - loss: -13.1460 - accuracy: 0.7464 - val_loss: -17.8603 - val_accuracy: 0.7143
Epoch 48/50
1/3 [=========>....................] - ETA: 0s - loss: -12.5047 - accuracy: 0.7472WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 7ms/step - loss: -13.7617 - accuracy: 0.7518 - val_loss: -18.7149 - val_accuracy: 0.7143
Epoch 49/50
1/3 [=========>....................] - ETA: 0s - loss: -12.9670 - accuracy: 0.7396WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 9ms/step - loss: -14.3190 - accuracy: 0.7464 - val_loss: -19.6643 - val_accuracy: 0.7143
Epoch 50/50
1/3 [=========>....................] - ETA: 0s - loss: -12.4192 - accuracy: 0.7358WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


3/3 [==============================] - 0s 10ms/step - loss: -14.1311 - accuracy: 0.7464 - val_loss: -20.7221 - val_accuracy: 0.7143
Model training finished


Validation accuracy: 71.43%
